# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\SymbolDigit_Processed_Files
df = pd.read_csv('SymbDigiCateg_Methylome_APPRDXFilter.csv')
print(df.shape)
df.head()

Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\SymbolDigit_Processed_Files
(458, 192588)


,Sentrix,PATNO,HYS,Gender,APPRDX,Symbol_Digit,cg20815302,cg25795414,cg00959375,cg07781847,...,cg03814399,cg14792798,cg16910097,cg08750857,cg17933893,cg06917935,cg02794319,cg10702286,cg10353174,cg19590050
0,200991620021_R01C01,3001,2,1,1,1,0.907957,0.868910,0.752844,0.827413,...,0.880496,0.928546,0.900567,0.877800,0.836823,0.216268,0.234350,0.681550,0.103758,0.837890
1,200973410167_R04C01,3007,2,1,1,2,0.918626,0.865257,0.798170,0.823846,...,0.877306,0.912956,0.935591,0.942104,0.750613,0.254265,0.290337,0.760554,0.075480,0.833824
2,200973410167_R05C01,3010,2,1,1,1,0.905297,0.864778,0.769766,0.803124,...,0.891733,0.902633,0.947783,0.942099,0.752775,0.229338,0.232169,0.775542,0.119109,0.846339
3,200989060001_R06C01,3012,2,1,1,1,0.944309,0.914462,0.808391,0.824763,...,0.906931,0.947272,0.914955,0.952388,0.914030,0.247900,0.266871,0.903613,0.077173,0.898589
4,200989060143_R08C01,3024,2,1,1,1,0.932802,0.900233,0.773236,0.815495,...,0.879621,0.945859,0.894625,0.942692,0.838772,0.183918,0.317935,0.871140,0.094540,0.863300


In [3]:
df1 = df.drop(['PATNO', 'Sentrix', 'HYS', 'Gender', 'Symbol_Digit'], axis=1)
df1.head(2)

,APPRDX,cg20815302,cg25795414,cg00959375,cg07781847,cg05138383,cg27394794,cg12443990,cg19788329,cg22178513,...,cg03814399,cg14792798,cg16910097,cg08750857,cg17933893,cg06917935,cg02794319,cg10702286,cg10353174,cg19590050
0,1,0.907957,0.868910,0.752844,0.827413,0.771393,0.766283,0.791932,0.785104,0.723989,...,0.880496,0.928546,0.900567,0.877800,0.836823,0.216268,0.234350,0.681550,0.103758,0.837890
1,1,0.918626,0.865257,0.798170,0.823846,0.760193,0.801117,0.772456,0.742722,0.580721,...,0.877306,0.912956,0.935591,0.942104,0.750613,0.254265,0.290337,0.760554,0.075480,0.833824


In [4]:
df1 = df1.transpose()
df1.head(2)

,0,1,2,3,4,5,6,7,8,9,...,448,449,450,451,452,453,454,455,456,457
APPRDX,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000,2.00000,2.000000,2.000000,2.000000
cg20815302,0.907957,0.918626,0.905297,0.944309,0.932802,0.925859,0.93942,0.920432,0.929847,0.923633,...,0.917489,0.938376,0.917218,0.936284,0.92572,0.930393,0.91248,0.917214,0.919081,0.920265


In [5]:
df1.rename(columns=df1.iloc[0], inplace = True)


In [6]:
df1.index.name ='CpG'
df1.reset_index(inplace=True)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,APPRDX,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000,2.00000,2.000000,2.000000,2.000000
1,cg20815302,0.907957,0.918626,0.905297,0.944309,0.932802,0.925859,0.93942,0.920432,0.929847,...,0.917489,0.938376,0.917218,0.936284,0.92572,0.930393,0.91248,0.917214,0.919081,0.920265


In [7]:
df1 = df1.drop ([0], axis = 0)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,cg20815302,0.907957,0.918626,0.905297,0.944309,0.932802,0.925859,0.939420,0.920432,0.929847,...,0.917489,0.938376,0.917218,0.936284,0.925720,0.930393,0.912480,0.917214,0.919081,0.920265
2,cg25795414,0.868910,0.865257,0.864778,0.914462,0.900233,0.889566,0.901912,0.856261,0.886372,...,0.887526,0.908529,0.875556,0.896841,0.824193,0.878968,0.863352,0.881230,0.880290,0.880562


In [8]:
df2 = pd.read_csv ("SymbDigi_NMI_Top50.csv")
df2.head(2)

,CpG
0,cg16272324
1,cg17937070


In [9]:
df3 = pd.merge (df1, df2, on= "CpG", how= "inner")
df3.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,cg16702987,0.691866,0.713975,0.681089,0.800989,0.715110,0.728480,0.736660,0.765335,0.698297,...,0.728574,0.748726,0.679383,0.76069,0.786642,0.744496,0.686777,0.707889,0.703662,0.682592
1,cg16916264,0.870274,0.927010,0.926633,0.937725,0.918665,0.933649,0.909331,0.925282,0.918833,...,0.901229,0.934339,0.911233,0.92431,0.867422,0.909184,0.881771,0.901983,0.911761,0.921459


In [10]:
df3 = df3.transpose()
df3.head(2)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
CpG,cg16702987,cg16916264,cg14547519,cg00508856,cg20230572,cg11169227,cg24402323,cg16448286,cg06538998,cg09879107,...,cg25361663,cg24323328,cg26022992,cg21899921,cg14547393,cg19056515,cg15469205,cg18813148,cg09826790,cg15942569
1.0,0.691866,0.870274,0.900845,0.903805,0.687517,0.887289,0.060656,0.906014,0.013463,0.727246,...,0.911043,0.024051,0.061537,0.032846,0.141759,0.966428,0.865868,0.0268,0.86927,0.888852


In [11]:
df3.rename(columns=df3.iloc[0], inplace = True)


In [12]:
df3.index.name ='APPRDX'
df3.reset_index(inplace=True)
df3.head(2)

,APPRDX,cg16702987,cg16916264,cg14547519,cg00508856,cg20230572,cg11169227,cg24402323,cg16448286,cg06538998,...,cg25361663,cg24323328,cg26022992,cg21899921,cg14547393,cg19056515,cg15469205,cg18813148,cg09826790,cg15942569
0,CpG,cg16702987,cg16916264,cg14547519,cg00508856,cg20230572,cg11169227,cg24402323,cg16448286,cg06538998,...,cg25361663,cg24323328,cg26022992,cg21899921,cg14547393,cg19056515,cg15469205,cg18813148,cg09826790,cg15942569
1,1.0,0.691866,0.870274,0.900845,0.903805,0.687517,0.887289,0.060656,0.906014,0.013463,...,0.911043,0.024051,0.061537,0.032846,0.141759,0.966428,0.865868,0.0268,0.86927,0.888852


In [13]:
df3 = df3.drop ([0], axis = 0)
df3.head(2)

,APPRDX,cg16702987,cg16916264,cg14547519,cg00508856,cg20230572,cg11169227,cg24402323,cg16448286,cg06538998,...,cg25361663,cg24323328,cg26022992,cg21899921,cg14547393,cg19056515,cg15469205,cg18813148,cg09826790,cg15942569
1,1.0,0.691866,0.870274,0.900845,0.903805,0.687517,0.887289,0.060656,0.906014,0.013463,...,0.911043,0.024051,0.061537,0.032846,0.141759,0.966428,0.865868,0.0268,0.86927,0.888852
2,1.0,0.713975,0.92701,0.902779,0.908528,0.741894,0.894105,0.024917,0.890996,0.010118,...,0.92591,0.02502,0.066522,0.046905,0.07878,0.976184,0.83076,0.020095,0.914964,0.884358


In [14]:
df3.reset_index(inplace = True, drop = True)
df3.head()

,APPRDX,cg16702987,cg16916264,cg14547519,cg00508856,cg20230572,cg11169227,cg24402323,cg16448286,cg06538998,...,cg25361663,cg24323328,cg26022992,cg21899921,cg14547393,cg19056515,cg15469205,cg18813148,cg09826790,cg15942569
0,1.0,0.691866,0.870274,0.900845,0.903805,0.687517,0.887289,0.060656,0.906014,0.013463,...,0.911043,0.024051,0.061537,0.032846,0.141759,0.966428,0.865868,0.0268,0.86927,0.888852
1,1.0,0.713975,0.92701,0.902779,0.908528,0.741894,0.894105,0.024917,0.890996,0.010118,...,0.92591,0.02502,0.066522,0.046905,0.07878,0.976184,0.83076,0.020095,0.914964,0.884358
2,1.0,0.681089,0.926633,0.893098,0.907461,0.765828,0.874573,0.024523,0.88818,0.010811,...,0.911595,0.023403,0.070549,0.041263,0.132671,0.971701,0.855777,0.018712,0.894834,0.882384
3,1.0,0.800989,0.937725,0.941512,0.909832,0.761336,0.875935,0.022393,0.916614,0.020383,...,0.962744,0.092731,0.041008,0.046077,0.148083,0.971067,0.834914,0.020647,0.92648,0.93742
4,1.0,0.71511,0.918665,0.93924,0.928716,0.700646,0.878241,0.022146,0.88896,0.006369,...,0.942566,0.014646,0.035192,0.034337,0.088535,0.977101,0.883551,0.025895,0.904223,0.933857


In [15]:
df3.shape

(458, 51)

In [16]:
df3 ['APPRDX'] = df3 ['APPRDX'].apply(int)


## RanFor 1000 times, CpG only 

In [17]:
df3.head(2)

,APPRDX,cg16702987,cg16916264,cg14547519,cg00508856,cg20230572,cg11169227,cg24402323,cg16448286,cg06538998,...,cg25361663,cg24323328,cg26022992,cg21899921,cg14547393,cg19056515,cg15469205,cg18813148,cg09826790,cg15942569
0,1,0.691866,0.870274,0.900845,0.903805,0.687517,0.887289,0.060656,0.906014,0.013463,...,0.911043,0.024051,0.061537,0.032846,0.141759,0.966428,0.865868,0.0268,0.86927,0.888852
1,1,0.713975,0.92701,0.902779,0.908528,0.741894,0.894105,0.024917,0.890996,0.010118,...,0.92591,0.02502,0.066522,0.046905,0.07878,0.976184,0.83076,0.020095,0.914964,0.884358


In [18]:
df3.shape

(458, 51)

In [19]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [20]:
dframe1.to_csv ("SymbDigi_NMITop50CpG_Accuracy_1000RF.csv")

In [21]:
dframe1.head(2)

,Accuracy,Std_Dev
0,0.675,0.092871
1,0.690,0.099499
